In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from gensim.models import KeyedVectors

In [3]:
ft=KeyedVectors.load("/content/drive/MyDrive/FYP/IMDb/FastText/ft")

In [4]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/drive/MyDrive/FYP/IMDb/IMDB-preprocessed.csv")
y=df["sentiment"]


In [5]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

2511


In [6]:
import tensorflow as tf
maxlen=66
max_words=len(ft.wv.vocab.keys())
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  tokenizer.fit_on_texts(X)
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])
X

array([[   19,  7608,  7149, ...,   123,  4013,   486],
       [    4,   203,     2, ...,  1900,    75,   216],
       [   10,   132,     3, ...,    67,    16,   333],
       ...,
       [ 3880,    70,   148, ..., 22558,     2,  5887],
       [   15,     1,   341, ...,    49,   695,    44],
       [    9,   154,    18, ...,   759,    11,    18]], dtype=int32)

In [7]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
for word,i in tokenizer.word_index.items():
  
  if word in ft and i < max_words:
    embed_vector=ft[word]
    
  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [8]:
embed_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.25959671,  0.26077929, -1.48361099, ..., -0.47585437,
         2.43393397, -2.77279496],
       [-0.19793859,  2.68988824, -3.07781458, ...,  0.221552  ,
        -0.36802348,  0.7608332 ],
       ...,
       [-0.23585528,  0.69668168, -0.91370511, ...,  0.51861876,
         0.71109408,  0.05964459],
       [-0.0630885 ,  1.14116943, -0.81346172, ..., -0.89304948,
        -0.65447277,  0.18485296],
       [-0.55161756,  0.52930117,  0.0418644 , ...,  1.35424447,
         0.99171054, -0.25952411]])

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [10]:
from tensorflow.keras import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout,Embedding,GRU,Dense,Flatten
model=Sequential()
model.add(Embedding(input_dim=max_words,output_dim=300,input_length=66,embeddings_initializer=Constant(embed_matrix)))
model.add(Flatten()) # loss stucks at about 
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(1,activation='sigmoid'))

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=1)

Epoch 1/100
586/586 [==============================] - 14s 18ms/step - loss: 0.7714 - accuracy: 0.5003
Epoch 2/100
586/586 [==============================] - 10s 18ms/step - loss: 0.7025 - accuracy: 0.5035
Epoch 3/100
586/586 [==============================] - 10s 18ms/step - loss: 0.6991 - accuracy: 0.5033
Epoch 4/100
586/586 [==============================] - 10s 18ms/step - loss: 0.7010 - accuracy: 0.5031
Epoch 5/100
586/586 [==============================] - 11s 19ms/step - loss: 0.6964 - accuracy: 0.5038
Epoch 6/100
586/586 [==============================] - 10s 18ms/step - loss: 0.7001 - accuracy: 0.5010
Epoch 7/100
586/586 [==============================] - 10s 18ms/step - loss: 0.6991 - accuracy: 0.5019
Epoch 8/100
586/586 [==============================] - 10s 18ms/step - loss: 0.6946 - accuracy: 0.5007
Epoch 9/100
586/586 [==============================] - 10s 18ms/step - loss: 0.6948 - accuracy: 0.5027
Epoch 10/100
586/586 [==============================] - 10s 18ms/step - l

In [12]:
y_pred=model.predict(X_test)
for i in range(len(y_pred)):
  if y_pred[i]>0.5:
    y_pred[i]=1
  else:
    y_pred[i]=0

In [13]:
#Metric calculation function
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))
display_metrics(y_test,y_pred)

0.78312
0.7812589031499371
0.7963827977106042
0.7846662676108433
